In [6]:
import os
import cv2
import pickle
import numpy as np
import face_recognition

In [7]:
# Read Pickle file
def readEncodingsPickle(fname):
    data = pickle.loads(open(fname, "rb").read())
    data = np.array(data)
    encodings = [d["encoding"] for d in data]
    names = [d["name"] for d in data]
    return encodings, names

# Create face encodings for a given image
def createEncodings(image):
    face_locations = face_recognition.face_locations(image)
    known_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)
    return known_encodings, face_locations

# Compare face encodings
def compareFaceEncodings(unknown_encoding, known_encodings, known_names):
    duplicateName = ""
    distance = 0.0
    matches = face_recognition.compare_faces(known_encodings, unknown_encoding, tolerance=0.5)
    face_distances = face_recognition.face_distance(known_encodings, unknown_encoding)
    best_match_index = np.argmin(face_distances)
    distance = face_distances[best_match_index]
    if matches[best_match_index]:
        acceptBool = True
        duplicateName = known_names[best_match_index]
    else:
        acceptBool = False
        duplicateName = ""
    return acceptBool, duplicateName, distance

# Save images to directory
def saveImageToDirectory(image, path, imageName):
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(os.path.join(path, imageName), image)

In [8]:
encodings_file = "./known_encodings.pickle"
people_encodings, names = readEncodingsPickle(encodings_file)
print(f"Loaded {len(people_encodings)} encodings from {encodings_file}")

Loaded 1434 encodings from ./known_encodings.pickle


In [12]:
def processNewImages(path):
    processed_images = set()  # to keep track of already processed images
    
    for img in os.listdir(path):
        imgPath = os.path.join(path, img)

        # Avoid duplicating images
        if imgPath in processed_images:
            continue
        processed_images.add(imgPath)

        # Read image
        image = cv2.imread(imgPath)
        if image is None:
            continue
        orig = image.copy()

        # Resize
        image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_LINEAR)

        # Get locations and encodings
        encs, locs = createEncodings(image)

        recognized_names = []
        unknown_count = 1

        # Processing image for each face
        for loc, unknown_encoding in zip(locs, encs):
            top, right, bottom, left = loc
            acceptBool, duplicateName, distance = compareFaceEncodings(unknown_encoding, people_encodings, names)
            if acceptBool:
                recognized_names.append(duplicateName)
            else:
                recognized_names.append(f"Unknown_{unknown_count}")
                unknown_count += 1
        
        # Group image based on recognized names
        num_people = len(recognized_names)
        if num_people > 1:
            if num_people == 2:
                group_name = f"group/2_people_group/{'+'.join(sorted(recognized_names))}"
            elif num_people == 3:
                group_name = f"group/3_people_group/{'+'.join(sorted(recognized_names))}"
            elif num_people == 4:
                group_name = f"group/4_people_group/{'+'.join(sorted(recognized_names))}"
            else:
                group_name = "Group"
            saveImageToDirectory(orig, f"./New_output/{group_name}", img)
        elif num_people == 1:
            saveImageToDirectory(orig, f"./New_output/{recognized_names[0]}", img)

        # Show Image
        for loc in locs:
            top, right, bottom, left = loc
            cv2.rectangle(image, (left, top), (right, bottom), color=(255, 0, 0), thickness=2)
            cv2.imshow("Image", image)
            cv2.waitKey(1)
            cv2.destroyAllWindows()

In [14]:
processNewImages(path="D:/Mommy/Internal/DCIM/Camera")
print("Processing Completed")

Processing Completed
